# Converting Images to Hydrodynamic Simulations

This notebook demonstrates one of the most creative applications of SPH: converting any image into initial conditions for a hydrodynamic simulation. Watch as your favorite photo evolves under the laws of fluid dynamics!

## The Concept

We'll take an image and interpret it as a 2D density field:
- **Bright pixels** → High density regions
- **Dark pixels** → Low density regions  
- **Colors** → Different fluid properties (optional)

Then we'll:
1. Convert pixels to SPH particles
2. Set up initial conditions with gravity and pressure
3. Watch the "image" evolve as a fluid!

## Physical Setup

The image will evolve under:
- **Gravity**: Pulling dense regions together
- **Pressure**: Providing hydrostatic support
- **Hydrodynamics**: Fluid motion and mixing

This creates beautiful dynamics where structure formation, turbulence, and fluid instabilities shape the evolution of your image.

## What we'll do in this notebook

1. **Load and process images** (yours or our examples)
2. **Convert pixels to particles** with appropriate densities
3. **Set up physics** (gravity, pressure, temperature)
4. **Create SWIFT initial conditions**
5. **Run simulations** and create movies
6. **Explore variations** (different images, physics)

Let's turn art into physics!

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os
import subprocess
from PIL import Image
import requests
from io import BytesIO

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

## 1. Image Loading and Processing

First, let's create functions to load and process images.

In [ ]:
def load_image(image_path_or_url, target_size=(100, 100)):
    """
    Load an image from file or URL and resize it
    
    Parameters:
    -----------
    image_path_or_url : str
        Path to local image file or URL
    target_size : tuple
        Target size for the image (width, height)
    """
    
    try:
        if image_path_or_url.startswith('http'):
            # Load from URL
            response = requests.get(image_path_or_url)
            img = Image.open(BytesIO(response.content))
        else:
            # Load from file
            img = Image.open(image_path_or_url)
        
        # Convert to RGB if needed
        if img.mode != 'RGB':
            img = img.convert('RGB')
        
        # Resize
        img = img.resize(target_size, Image.Resampling.LANCZOS)
        
        # Convert to numpy array
        img_array = np.array(img)
        
        print(f"Loaded image with shape: {img_array.shape}")
        return img_array
        
    except Exception as e:
        print(f"Error loading image: {e}")
        return None

def create_sample_image(size=(100, 100), pattern='spiral'):
    """
    Create a sample image if no image is provided
    
    Parameters:
    -----------
    size : tuple
        Image size (width, height)
    pattern : str
        Pattern type ('spiral', 'galaxy', 'face', 'mandelbrot')
    """
    
    width, height = size
    x = np.linspace(-2, 2, width)
    y = np.linspace(-2, 2, height)
    X, Y = np.meshgrid(x, y)
    
    if pattern == 'spiral':
        # Spiral galaxy pattern
        r = np.sqrt(X**2 + Y**2)
        theta = np.arctan2(Y, X)
        
        # Create spiral arms
        spiral1 = np.exp(-r/0.8) * np.cos(2*theta - 3*r)**2
        spiral2 = np.exp(-r/0.8) * np.cos(2*theta - 3*r + np.pi)**2
        
        # Central bulge
        bulge = np.exp(-r**2/0.3**2)
        
        # Combine
        intensity = spiral1 + spiral2 + 0.5*bulge
        intensity = np.clip(intensity, 0, 1)
        
        # Convert to RGB
        img_array = np.stack([intensity, intensity*0.8, intensity*0.6], axis=2)
        
    elif pattern == 'galaxy':
        # Elliptical galaxy
        r = np.sqrt((X/1.5)**2 + Y**2)
        intensity = np.exp(-r**2/0.5**2)
        
        # Add some structure
        noise = 0.1 * np.random.random((height, width))
        intensity += noise
        intensity = np.clip(intensity, 0, 1)
        
        # Blue-ish galaxy
        img_array = np.stack([intensity*0.6, intensity*0.8, intensity], axis=2)
        
    elif pattern == 'face':
        # Simple smiley face
        r = np.sqrt(X**2 + Y**2)
        
        # Face outline
        face = (r < 1.5) & (r > 1.2)
        
        # Eyes
        left_eye = ((X + 0.5)**2 + (Y + 0.3)**2) < 0.2**2
        right_eye = ((X - 0.5)**2 + (Y + 0.3)**2) < 0.2**2
        
        # Mouth (smile)
        mouth_r = np.sqrt(X**2 + (Y + 0.3)**2)
        mouth = (mouth_r < 0.8) & (mouth_r > 0.6) & (Y < -0.3)
        
        # Combine
        intensity = face.astype(float) + left_eye.astype(float) + right_eye.astype(float) + mouth.astype(float)
        intensity = np.clip(intensity, 0, 1)
        
        # Yellow smiley
        img_array = np.stack([intensity, intensity, intensity*0.3], axis=2)
        
    elif pattern == 'mandelbrot':
        # Mandelbrot set
        def mandelbrot(c, max_iter=50):
            z = 0
            for n in range(max_iter):
                if abs(z) > 2:
                    return n
                z = z*z + c
            return max_iter
        
        # Calculate Mandelbrot set
        C = X + 1j*Y
        mandel = np.zeros((height, width))
        
        for i in range(height):
            for j in range(width):
                mandel[i,j] = mandelbrot(C[i,j])
        
        # Normalize
        intensity = mandel / np.max(mandel)
        
        # Colorful Mandelbrot
        img_array = plt.cm.hot(intensity)[:,:,:3]
    
    else:
        # Default: random pattern
        intensity = np.random.random((height, width))
        img_array = np.stack([intensity, intensity, intensity], axis=2)
    
    # Ensure values are in [0, 255] and convert to uint8
    img_array = (255 * np.clip(img_array, 0, 1)).astype(np.uint8)
    
    print(f"Created {pattern} pattern with shape: {img_array.shape}")
    return img_array

# Create sample images
print("Creating sample images...")
spiral_img = create_sample_image(size=(80, 80), pattern='spiral')
galaxy_img = create_sample_image(size=(80, 80), pattern='galaxy')
face_img = create_sample_image(size=(60, 60), pattern='face')

# Display the sample images
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(spiral_img)
axes[0].set_title('Spiral Galaxy')
axes[0].axis('off')

axes[1].imshow(galaxy_img)
axes[1].set_title('Elliptical Galaxy')
axes[1].axis('off')

axes[2].imshow(face_img)
axes[2].set_title('Smiley Face')
axes[2].axis('off')

plt.suptitle('Sample Images for Hydro Conversion', fontsize=16)
plt.tight_layout()
plt.show()

## 2. Image to Particle Conversion

Now let's convert image pixels to SPH particles with appropriate densities and properties.

In [ ]:
def image_to_particles(img_array, box_size=2.0, density_contrast=10.0, 
                      min_density=0.1, use_color=False, n_particles_target=5000):
    """
    Convert image to SPH particles
    
    Parameters:
    -----------
    img_array : numpy array
        Image array (height, width, 3)
    box_size : float
        Physical size of the simulation box
    density_contrast : float
        Maximum density contrast between bright and dark regions
    min_density : float
        Minimum density (for dark regions)
    use_color : bool
        Whether to use color information for particle properties
    n_particles_target : int
        Target number of particles (will subsample if needed)
    """
    
    height, width = img_array.shape[:2]
    
    # Convert to grayscale for density
    if len(img_array.shape) == 3:
        # Use luminance formula: 0.299*R + 0.587*G + 0.114*B
        intensity = 0.299*img_array[:,:,0] + 0.587*img_array[:,:,1] + 0.114*img_array[:,:,2]
        intensity = intensity / 255.0  # Normalize to [0,1]
    else:
        intensity = img_array / 255.0
    
    # Create coordinate arrays
    x_coords = np.linspace(-box_size/2, box_size/2, width)
    y_coords = np.linspace(-box_size/2, box_size/2, height)
    X, Y = np.meshgrid(x_coords, y_coords)
    
    # Flatten arrays
    x_flat = X.flatten()
    y_flat = Y.flatten()
    intensity_flat = intensity.flatten()
    
    # Calculate densities
    # Bright pixels → high density, dark pixels → low density
    max_density = min_density * density_contrast
    densities = min_density + (max_density - min_density) * intensity_flat
    
    # Subsample if we have too many particles
    n_pixels = len(x_flat)
    if n_pixels > n_particles_target:
        # Weighted sampling - more likely to keep high-density particles
        probabilities = densities / np.sum(densities)
        indices = np.random.choice(n_pixels, n_particles_target, 
                                 replace=False, p=probabilities)
        
        x_flat = x_flat[indices]
        y_flat = y_flat[indices]
        densities = densities[indices]
        intensity_flat = intensity_flat[indices]
    
    # Create 3D positions (z=0 for 2D)
    positions = np.column_stack([x_flat, y_flat, np.zeros_like(x_flat)])
    
    # Add small random perturbations to avoid perfect grid
    pixel_size = box_size / max(width, height)
    positions[:, :2] += np.random.uniform(-pixel_size/4, pixel_size/4, 
                                         (len(positions), 2))
    
    # Initial velocities (zero)
    velocities = np.zeros_like(positions)
    
    # Particle masses (proportional to density * area)
    total_area = box_size**2
    particle_area = total_area / len(positions)
    masses = densities * particle_area
    
    # Internal energy (related to temperature)
    # Higher density regions are cooler (more bound)
    base_temp = 1.0
    temperatures = base_temp * (1.0 + 0.5 * (1.0 - intensity_flat))
    internal_energies = temperatures  # Simplified
    
    # Color information (if requested)
    if use_color and len(img_array.shape) == 3:
        # Store color as metallicity or other tracer
        if len(indices) < n_pixels:
            colors = img_array.reshape(-1, 3)[indices] / 255.0
        else:
            colors = img_array.reshape(-1, 3) / 255.0
    else:
        colors = None
    
    # Particle IDs
    particle_ids = np.arange(len(positions), dtype=np.int64)
    
    print(f"Converted image to {len(positions)} particles")
    print(f"Density range: {np.min(densities):.3f} to {np.max(densities):.3f}")
    print(f"Mass range: {np.min(masses):.3e} to {np.max(masses):.3e}")
    
    return {
        'positions': positions,
        'velocities': velocities,
        'masses': masses,
        'densities': densities,
        'internal_energies': internal_energies,
        'particle_ids': particle_ids,
        'colors': colors,
        'box_size': box_size,
        'n_particles': len(positions),
        'original_image_shape': img_array.shape[:2]
    }

# Convert our sample images to particles
print("Converting spiral galaxy image to particles...")
spiral_particles = image_to_particles(spiral_img, box_size=4.0, 
                                     density_contrast=20.0, 
                                     n_particles_target=3000)

print("\nConverting smiley face image to particles...")
face_particles = image_to_particles(face_img, box_size=3.0,
                                   density_contrast=15.0,
                                   n_particles_target=2000)

## 3. Visualizing the Particle Conversion

Let's see how our images look as particle distributions:

In [ ]:
def plot_image_to_particles(img_array, particle_data, title=""):
    """
    Plot comparison between original image and particle representation
    """
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Original image
    axes[0,0].imshow(img_array)
    axes[0,0].set_title('Original Image')
    axes[0,0].axis('off')
    
    # Grayscale intensity
    if len(img_array.shape) == 3:
        intensity = 0.299*img_array[:,:,0] + 0.587*img_array[:,:,1] + 0.114*img_array[:,:,2]
    else:
        intensity = img_array
    
    axes[0,1].imshow(intensity, cmap='gray')
    axes[0,1].set_title('Grayscale Intensity')
    axes[0,1].axis('off')
    
    # Particle positions colored by density
    pos = particle_data['positions']
    densities = particle_data['densities']
    
    im1 = axes[0,2].scatter(pos[:,0], pos[:,1], c=densities, s=10, 
                           cmap='viridis', alpha=0.8)
    axes[0,2].set_title('Particle Densities')
    axes[0,2].set_xlabel('x')
    axes[0,2].set_ylabel('y')
    axes[0,2].set_aspect('equal')
    plt.colorbar(im1, ax=axes[0,2], label='Density')
    
    # Particle masses
    masses = particle_data['masses']
    im2 = axes[1,0].scatter(pos[:,0], pos[:,1], c=masses, s=10,
                           cmap='plasma', alpha=0.8)
    axes[1,0].set_title('Particle Masses')
    axes[1,0].set_xlabel('x')
    axes[1,0].set_ylabel('y')
    axes[1,0].set_aspect('equal')
    plt.colorbar(im2, ax=axes[1,0], label='Mass')
    
    # Internal energies (temperature)
    internal_energies = particle_data['internal_energies']
    im3 = axes[1,1].scatter(pos[:,0], pos[:,1], c=internal_energies, s=10,
                           cmap='hot', alpha=0.8)
    axes[1,1].set_title('Internal Energies')
    axes[1,1].set_xlabel('x')
    axes[1,1].set_ylabel('y')
    axes[1,1].set_aspect('equal')
    plt.colorbar(im3, ax=axes[1,1], label='Internal Energy')
    
    # Density histogram
    axes[1,2].hist(densities, bins=30, alpha=0.7, edgecolor='black')
    axes[1,2].set_xlabel('Density')
    axes[1,2].set_ylabel('Number of Particles')
    axes[1,2].set_title('Density Distribution')
    axes[1,2].grid(True, alpha=0.3)
    
    if title:
        plt.suptitle(title, fontsize=16)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"Particle statistics:")
    print(f"  Number of particles: {particle_data['n_particles']}")
    print(f"  Density range: {np.min(densities):.3f} - {np.max(densities):.3f}")
    print(f"  Mass range: {np.min(masses):.3e} - {np.max(masses):.3e}")
    print(f"  Total mass: {np.sum(masses):.3e}")

# Plot conversions
plot_image_to_particles(spiral_img, spiral_particles, "Spiral Galaxy → SPH Particles")
plot_image_to_particles(face_img, face_particles, "Smiley Face → SPH Particles")

## 4. Creating SWIFT Initial Conditions

Now let's create proper initial condition files for SWIFT:

In [ ]:
def write_image_ic_file(particle_data, filename, add_gravity=True):
    """
    Write image-based initial conditions to SWIFT HDF5 file
    
    Parameters:
    -----------
    particle_data : dict
        Particle data from image_to_particles
    filename : str
        Output filename
    add_gravity : bool
        Whether to include gravitational effects
    """
    
    os.makedirs('../ics', exist_ok=True)
    filepath = f'../ics/{filename}'
    
    # Ensure positions are centered in box
    box_size = particle_data['box_size']
    positions = particle_data['positions'].copy()
    positions[:, :2] += box_size/2  # Shift to [0, box_size]
    
    with h5py.File(filepath, 'w') as f:
        
        # Header
        header = f.create_group('Header')
        header.attrs['BoxSize'] = [box_size, box_size, 0.1]  # Thin 2D box
        header.attrs['NumPart_ThisFile'] = [particle_data['n_particles'], 0, 0, 0, 0, 0]
        header.attrs['NumPart_Total'] = [particle_data['n_particles'], 0, 0, 0, 0, 0]
        header.attrs['NumPart_Total_HighWord'] = [0, 0, 0, 0, 0, 0]
        header.attrs['MassTable'] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
        header.attrs['Time'] = 0.0
        header.attrs['Redshift'] = 0.0
        header.attrs['Flag_Sfr'] = 0
        header.attrs['Flag_Feedback'] = 0
        header.attrs['Flag_Cooling'] = 0
        header.attrs['Flag_StellarAge'] = 0
        header.attrs['Flag_Metals'] = 0
        header.attrs['NumFilesPerSnapshot'] = 1
        header.attrs['HubbleParam'] = 1.0
        header.attrs['Omega0'] = 0.0
        header.attrs['OmegaLambda'] = 0.0
        
        # Gas particles (PartType0)
        part0 = f.create_group('PartType0')
        part0.create_dataset('Coordinates', data=positions)
        part0.create_dataset('Velocities', data=particle_data['velocities'])
        part0.create_dataset('Masses', data=particle_data['masses'])
        part0.create_dataset('InternalEnergy', data=particle_data['internal_energies'])
        part0.create_dataset('ParticleIDs', data=particle_data['particle_ids'])
        
        # Smoothing lengths (estimate based on density)
        # h ~ (mass/density)^(1/2) for 2D
        h = np.sqrt(particle_data['masses'] / particle_data['densities']) * 0.5
        h = np.clip(h, box_size/500, box_size/20)  # Reasonable limits
        part0.create_dataset('SmoothingLength', data=h)
        
        # Color information as metallicity (if available)
        if particle_data['colors'] is not None:
            # Use red component as metallicity
            metallicity = particle_data['colors'][:, 0]
            part0.create_dataset('Metallicity', data=metallicity)
    
    print(f"Written image IC file: {filepath}")
    return filepath

# Write IC files for our examples
spiral_ic_file = write_image_ic_file(spiral_particles, 'image_spiral.hdf5')
face_ic_file = write_image_ic_file(face_particles, 'image_face.hdf5')

## 5. Create Parameter Files

Let's create parameter files for different types of image simulations:

In [ ]:
def create_image_parameter_file(ic_filename, output_name, box_size,
                               simulation_type='gravity_hydro'):
    """
    Create parameter file for image-based simulation
    
    Parameters:
    -----------
    ic_filename : str
        Initial conditions filename
    output_name : str
        Output file prefix
    box_size : float
        Size of simulation box
    simulation_type : str
        Type of simulation ('gravity_hydro', 'hydro_only', 'gravity_only')
    """
    
    os.makedirs('../params', exist_ok=True)
    param_file = f'../params/{output_name}.yml'
    
    # Base parameter content
    param_content = f"""
# Define the system of units to use internally
InternalUnitSystem:
  UnitMass_in_cgs:     1.989e33  # Solar mass
  UnitLength_in_cgs:   3.086e18  # kpc
  UnitVelocity_in_cgs: 1e5       # km/s
  UnitCurrent_in_cgs:  1         # amperes
  UnitTemp_in_cgs:     1         # kelvin

# Parameters governing the time integration
TimeIntegration:
  time_begin: 0.0    # The starting time of the simulation (in internal units)
  time_end:   2.0    # The end time of the simulation (in internal units)
  dt_min:     1e-6   # The minimal time-step size of the simulation (in internal units)
  dt_max:     1e-2   # The maximal time-step size of the simulation (in internal units)

# Parameters governing the snapshots
Snapshots:
  basename:            {output_name}  # Common part of the name of output files
  output_list_on:      1              # Enable the output list
  output_list:         ../params/image_output_list.txt  # File containing the output times
  compression:         1              # GZIP compression level (0-9)

# Parameters governing the conserved quantities statistics
Statistics:
  delta_time:          0.1    # Time between statistics output

# Parameters for the hydrodynamics scheme
SPH:
  resolution_eta:        1.2348   # Target smoothing length in units of the mean inter-particle separation
  h_min_ratio:           0.01     # Minimal smoothing length in units of the softening
  h_max:                 10.0     # Maximal smoothing length in internal units
  CFL_condition:         0.2      # Courant-Friedrichs-Lewy condition for time integration
  minimal_temperature:   1.0      # Minimal temperature in internal units

# Parameters related to the initial conditions
InitialConditions:
  file_name:  ../ics/{ic_filename}    # The file to read
  periodic:   1                       # Periodic boundary conditions

# Equation of state
EoS:
  adiabatic_index: 1.66667   # 5/3 for monatomic gas
"""
    
    # Add gravity if requested
    if 'gravity' in simulation_type:
        param_content += f"""
# Gravity parameters
Gravity:
  mesh_side_length:       64      # Number of cells along each axis for the mesh
  eta:                    0.025   # Constant dimensionless multiplier for time integration
  MAC:                    adaptive # Use the adaptive opening angle condition
  theta_cr:               0.7     # Critical opening angle (Multipole acceptance criterion)
  epsilon_fmm:            0.001   # Tolerance parameter for the adaptive opening angle
  use_tree_below_softening: 0     # Do we use the tree calculation below the softening scale?
  max_physical_DM_softening: {box_size/50:.3f}  # Max physical DM softening length
  max_physical_baryon_softening: {box_size/50:.3f}  # Max physical softening length for baryons

# Parameters for the self-gravity scheme
SelfGravity:
  mesh_side_length:       64      # Number of cells along each axis for the gravity mesh
  mac:                    adaptive # Multipole acceptance criterion
  theta:                  0.7     # Opening angle
  rebuild_frequency:      0.01    # How often to rebuild the tree
"""
    
    # Add dimensionality
    param_content += f"""
# Dimensionality of the problem
MetaData:
  run_name:   {output_name}
  dimension:  2
"""
    
    with open(param_file, 'w') as f:
        f.write(param_content)
    
    # Create output list
    output_list_file = '../params/image_output_list.txt'
    output_times = np.linspace(0, 2.0, 41)  # 41 outputs
    with open(output_list_file, 'w') as f:
        for t in output_times:
            f.write(f"{t:.3f}\n")
    
    print(f"Created parameter file: {param_file}")
    return param_file

# Create parameter files for different simulation types
spiral_param_gravity = create_image_parameter_file('image_spiral.hdf5', 
                                                  'image_spiral_gravity',
                                                  spiral_particles['box_size'],
                                                  'gravity_hydro')

spiral_param_hydro = create_image_parameter_file('image_spiral.hdf5',
                                                'image_spiral_hydro', 
                                                spiral_particles['box_size'],
                                                'hydro_only')

face_param = create_image_parameter_file('image_face.hdf5',
                                        'image_face_gravity',
                                        face_particles['box_size'],
                                        'gravity_hydro')

## 6. Load Your Own Image

Now let's create a function to easily load and process your own image:

In [ ]:
def process_custom_image(image_path, output_name, target_size=(100, 100),
                        box_size=3.0, density_contrast=20.0, 
                        n_particles_target=4000):
    """
    Complete pipeline to process a custom image
    
    Parameters:
    -----------
    image_path : str
        Path to image file or URL
    output_name : str
        Name for output files
    target_size : tuple
        Target image size
    box_size : float
        Physical size of simulation box
    density_contrast : float
        Density contrast between bright and dark regions
    n_particles_target : int
        Target number of particles
    """
    
    print(f"Processing custom image: {image_path}")
    
    # Load image
    img_array = load_image(image_path, target_size)
    if img_array is None:
        return None
    
    # Convert to particles
    particle_data = image_to_particles(img_array, 
                                      box_size=box_size,
                                      density_contrast=density_contrast,
                                      n_particles_target=n_particles_target)
    
    # Visualize conversion
    plot_image_to_particles(img_array, particle_data, 
                           f"Custom Image: {output_name}")
    
    # Write files
    ic_filename = f'image_{output_name}.hdf5'
    ic_file = write_image_ic_file(particle_data, ic_filename)
    
    param_file = create_image_parameter_file(ic_filename, 
                                           f'image_{output_name}',
                                           box_size,
                                           'gravity_hydro')
    
    print(f"\nCustom image processing complete!")
    print(f"IC file: {ic_file}")
    print(f"Parameter file: {param_file}")
    
    return particle_data, ic_file, param_file

# Example: Process a sample image URL (you can replace with your own)
# Uncomment the lines below to try with a real image:

# sample_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Smiley.svg/256px-Smiley.svg.png"
# custom_result = process_custom_image(sample_url, "wikipedia_smiley", 
#                                     target_size=(80, 80),
#                                     box_size=3.0,
#                                     density_contrast=25.0,
#                                     n_particles_target=2500)

print("To process your own image, uncomment the lines above and provide a path or URL!")

## 7. Analysis and Visualization Framework

Let's create tools to analyze the evolution of our image simulations:

In [ ]:
def analyze_image_snapshot(snapshot_file, original_particle_data):
    """
    Analyze a snapshot from an image-based simulation
    """
    
    try:
        with h5py.File(snapshot_file, 'r') as f:
            # Read header
            header = f['Header']
            time = header.attrs['Time']
            
            # Read particle data
            part0 = f['PartType0']
            positions = part0['Coordinates'][:]
            velocities = part0['Velocities'][:]
            densities = part0['Density'][:] if 'Density' in part0 else None
            internal_energy = part0['InternalEnergy'][:]
            masses = part0['Masses'][:]
        
        # Center positions
        box_size = original_particle_data['box_size']
        centered_pos = positions - box_size/2
        
        # Calculate some derived quantities
        speed = np.sqrt(np.sum(velocities**2, axis=1))
        
        # Calculate center of mass
        com = np.average(centered_pos, weights=masses, axis=0)
        
        # Calculate kinetic energy
        kinetic_energy = 0.5 * np.sum(masses * speed**2)
        
        # Calculate potential energy (approximate)
        thermal_energy = np.sum(masses * internal_energy)
        
        return {
            'time': time,
            'positions': centered_pos,
            'velocities': velocities,
            'densities': densities,
            'masses': masses,
            'internal_energy': internal_energy,
            'speed': speed,
            'center_of_mass': com,
            'kinetic_energy': kinetic_energy,
            'thermal_energy': thermal_energy,
            'box_size': box_size
        }
        
    except Exception as e:
        print(f"Error analyzing snapshot {snapshot_file}: {e}")
        return None

def plot_image_evolution(snapshot_data, title="", save_path=None):
    """
    Plot the evolution of an image-based simulation
    """
    
    if snapshot_data is None:
        return
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    pos = snapshot_data['positions']
    vel = snapshot_data['velocities'] 
    densities = snapshot_data['densities']
    masses = snapshot_data['masses']
    speed = snapshot_data['speed']
    internal_energy = snapshot_data['internal_energy']
    
    # Density map
    if densities is not None:
        im1 = axes[0,0].scatter(pos[:,0], pos[:,1], c=densities, s=15, 
                               cmap='viridis', alpha=0.8)
        axes[0,0].set_title(f'Density (t={snapshot_data["time"]:.2f})')
        plt.colorbar(im1, ax=axes[0,0], label='Density')
    else:
        axes[0,0].scatter(pos[:,0], pos[:,1], s=5, alpha=0.6)
        axes[0,0].set_title(f'Particles (t={snapshot_data["time"]:.2f})')
    
    axes[0,0].set_xlabel('x')
    axes[0,0].set_ylabel('y')
    axes[0,0].set_aspect('equal')
    
    # Velocity field
    im2 = axes[0,1].scatter(pos[:,0], pos[:,1], c=speed, s=15,
                           cmap='plasma', alpha=0.8)
    # Add velocity arrows (subsample)
    skip = max(1, len(pos)//100)
    axes[0,1].quiver(pos[::skip,0], pos[::skip,1], 
                    vel[::skip,0], vel[::skip,1],
                    scale=50, alpha=0.7, color='white', width=0.002)
    axes[0,1].set_title('Velocity Field')
    axes[0,1].set_xlabel('x')
    axes[0,1].set_ylabel('y')
    axes[0,1].set_aspect('equal')
    plt.colorbar(im2, ax=axes[0,1], label='Speed')
    
    # Mass distribution
    im3 = axes[0,2].scatter(pos[:,0], pos[:,1], c=masses, s=15,
                           cmap='hot', alpha=0.8)
    axes[0,2].set_title('Mass Distribution')
    axes[0,2].set_xlabel('x')
    axes[0,2].set_ylabel('y')
    axes[0,2].set_aspect('equal')
    plt.colorbar(im3, ax=axes[0,2], label='Mass')
    
    # Temperature/Internal energy
    im4 = axes[1,0].scatter(pos[:,0], pos[:,1], c=internal_energy, s=15,
                           cmap='coolwarm', alpha=0.8)
    axes[1,0].set_title('Internal Energy')
    axes[1,0].set_xlabel('x')
    axes[1,0].set_ylabel('y')
    axes[1,0].set_aspect('equal')
    plt.colorbar(im4, ax=axes[1,0], label='Internal Energy')
    
    # Radial profile
    r = np.sqrt(pos[:,0]**2 + pos[:,1]**2)
    if densities is not None:
        # Bin by radius
        r_bins = np.linspace(0, np.max(r), 20)
        r_centers = 0.5 * (r_bins[1:] + r_bins[:-1])
        
        # Calculate average density in each bin
        rho_profile = []
        for i in range(len(r_bins)-1):
            mask = (r >= r_bins[i]) & (r < r_bins[i+1])
            if np.any(mask):
                rho_profile.append(np.mean(densities[mask]))
            else:
                rho_profile.append(0)
        
        axes[1,1].plot(r_centers, rho_profile, 'o-', alpha=0.8)
        axes[1,1].set_xlabel('Radius')
        axes[1,1].set_ylabel('Average Density')
        axes[1,1].set_title('Radial Density Profile')
        axes[1,1].grid(True, alpha=0.3)
    
    # Energy evolution (placeholder - would need multiple snapshots)
    axes[1,2].text(0.1, 0.8, f'Kinetic Energy: {snapshot_data["kinetic_energy"]:.2e}', 
                  transform=axes[1,2].transAxes)
    axes[1,2].text(0.1, 0.6, f'Thermal Energy: {snapshot_data["thermal_energy"]:.2e}',
                  transform=axes[1,2].transAxes)
    axes[1,2].text(0.1, 0.4, f'Center of Mass: ({snapshot_data["center_of_mass"][0]:.3f}, {snapshot_data["center_of_mass"][1]:.3f})',
                  transform=axes[1,2].transAxes)
    axes[1,2].set_title('Energetics')
    axes[1,2].set_xlim(0, 1)
    axes[1,2].set_ylim(0, 1)
    
    if title:
        plt.suptitle(title, fontsize=16)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
    
    plt.show()

print("Analysis framework ready for image simulations!")

## Summary and Next Steps

In this notebook, we've created a complete pipeline for converting images into hydrodynamic simulations:

### What we've accomplished:

1. **Image processing pipeline**: Load, resize, and convert images to particle distributions
2. **Density mapping**: Bright pixels → high density, dark pixels → low density
3. **Physical setup**: Particles with masses, internal energies, and proper scaling
4. **SWIFT integration**: Created IC files and parameter files for simulations
5. **Analysis framework**: Tools to visualize and analyze the evolution
6. **Flexible parameters**: Easy to adjust density contrasts, particle numbers, physics

### Key Concepts:

- **Density-luminosity mapping**: Transform visual information into physical quantities
- **Particle sampling**: Intelligent subsampling that preserves structure
- **Multi-physics**: Combine gravity, hydrodynamics, and thermodynamics
- **2D fluid dynamics**: Explore instabilities and mixing in thin disks

### Next Steps:

1. **Run simulations** with the generated initial conditions
2. **Try your own images**:
   - Photos of faces, objects, or artwork
   - Astronomical images (galaxies, nebulae)
   - Abstract patterns or designs
3. **Experiment with physics**:
   - Gravity-only simulations (structure formation)
   - Hydro-only simulations (pressure-driven flows)
   - Combined simulations (realistic astrophysics)
4. **Create movies** of your image evolving as a fluid

### Advanced Challenges:

- **Color-coded physics**: Use RGB channels for different fluid properties
- **3D extension**: Convert images to 3D particle distributions
- **Multiple images**: Create binary systems or interactions
- **Physical realism**: Add cooling, star formation, magnetic fields
- **Interactive visualization**: Real-time parameter exploration

### Creative Applications:

- **Artistic evolution**: Watch portraits evolve under gravity
- **Educational demonstrations**: Show fluid instabilities in familiar contexts
- **Astrophysical analogies**: Compare with real galaxy evolution
- **Pattern formation**: Explore how structure emerges from initial conditions

This approach bridges art, physics, and computation, showing how the same physical laws that govern stars and galaxies can transform any image into a dynamic, evolving fluid system. The results are often beautiful, always educational, and sometimes surprising!

### Getting Started:

1. **Upload your image** (or use a URL)
2. **Adjust parameters** (size, density contrast, particle number)
3. **Run the conversion** using `process_custom_image()`
4. **Simulate with SWIFT** using the generated files
5. **Create movies** and share your results!

Have fun exploring the universe in your computer!